In [ ]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
from skimage.color import gray2rgb
from tqdm import tqdm
import tifffile
from PIL import Image
import json

In [ ]:
adata = sc.read("../../data/xenium_outs/merged_processed_integrated.h5ad")

In [ ]:
raw_dir = "../../data/raw"
sample_folders = [os.path.join(raw_dir, folder) for folder in os.listdir(raw_dir) if "human_ANCA" in folder]

In [ ]:
xenium_folders_0 = []
ids_folders = []
for folder in tqdm(sample_folders):
    tmp = os.listdir(folder)
    for folder_tmp in tmp:
        if "output-XET" in folder_tmp:
            xenium_folders_0.append(os.path.join(folder, folder_tmp))
            ids_folders.append(folder_tmp.split("_")[2])

In [ ]:
ids_folders = [e for e in ids_folders if e in adata.obs.Slide_ID.unique()]

In [ ]:
xenium_folders = []
for i in ids_folders:
    for f in xenium_folders_0:
        if i in f:
            xenium_folders.append(f)

In [ ]:
for i in range(len(ids_folders)):
    print(ids_folders[i], xenium_folders[i])

In [ ]:
print(len(ids_folders))

In [ ]:
LEVEL = 1
px_size = 0.425

In [ ]:
"spatial" in adata.uns.keys()

In [ ]:
adata.uns["spatial"] = {}

In [ ]:
for i, ID in tqdm(enumerate(ids_folders)):
    print(ID)
    mipurl = os.path.join(xenium_folders[i], "morphology_mip.ome.tif")
    print("Processing DAPI")
    with tifffile.TiffFile(mipurl) as tif:
        image = tif.series[0].levels[LEVEL].asarray()

    resample = scipy.ndimage.zoom(image, px_size)
    resample_invert = np.invert(resample)
    resample_invert = (resample_invert-resample_invert.min())/(resample_invert.max()-resample_invert.min())
    rgb = gray2rgb(resample_invert)
        
    print("Adding DAPI to anndata")

    adata.uns["spatial"][f'{ID}'] = {}
    adata.uns["spatial"][f'{ID}']["images"] = {}
    adata.uns["spatial"][f'{ID}']["scalefactors"] = {}
        
    adata.uns["spatial"][f'{ID}']["images"][f"dapi_LEVEL{LEVEL}"] = rgb
    adata.uns["spatial"][f'{ID}']["scalefactors"][f"tissue_dapi_LEVEL{LEVEL}_scalef"] = 1
    adata.uns["spatial"][f'{ID}']["scalefactors"]["fiducial_diameter_fullres"] = 40
    adata.uns["spatial"][f'{ID}']["scalefactors"]["spot_diameter_fullres"] = 40

In [ ]:
sc.pl.spatial(adata[adata.obs.Slide_ID==ids_folders[0]], library_id=ids_folders[0], color="CD3D", img_key=f"dapi_LEVEL{LEVEL}")

In [ ]:
sc.pl.spatial(adata[adata.obs.Slide_ID==ids_folders[0]], library_id=ids_folders[0], img_key=f"dapi_LEVEL{LEVEL}")


In [ ]:
adata.write("../../data/xenium_outs/merged_processed_integrated_v2.h5ad")